# Waffle House Formal Authentication Tokenizer Concept


In [1]:
# Import libraries

import pandas as pd
from huggingface_hub import notebook_login
from pprint import pprint
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# Login to hugging face
notebook_login()

In [3]:
# Import tokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

C:\Users\Ryan Levey\OneDrive\Bureaublad\vandy_cs\CS 6315\final_project\waffle_house_project\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ryan Levey\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [7]:
# Tokenized Text Example

text = "Hello how are you"

In [5]:
# Passing thru the tokenizer

tokenized_text = tokenizer(text)["input_ids"]

In [6]:
# Print tokens

tokenized_text

[101, 8667, 1293, 1132, 1128, 102]

In [9]:
# Illustrating tokenized text

untokenized_text = tokenizer.decode(tokenized_text)

untokenized_text


'[CLS] Hello how are you [SEP]'

## Waffle House Tokenization Example

In [11]:
# Waffle House Token Example

waffle_tokens = {
    "jelly_bottom": "scrambled_eggs",
    "hashbrowns_top": "hashbrowns_selected",
    "tomato_top": "tomato_selected",
    "napkin_top": "oatmeal_selected"
}

In [12]:
# Waffle House Tokenize Authentication Phrase

tokenized_markers = {k: tokenizer(v)["input_ids"] for k, v in waffle_tokens.items()}
pprint(tokenized_markers)


In [13]:
def authenticate_user(tokens):
    for token, expected_token in tokenized_markers.items():
        if tokens.get(token) != expected_token:
            return False
    return True


In [14]:
# Sample interaction

user_tokens = {
    "jelly_bottom": tokenizer("scrambled_eggs")["input_ids"],
    "hashbrowns_top": tokenizer("hashbrowns_selected")["input_ids"]
}
auth_status = authenticate_user(user_tokens)
print("Access Granted" if auth_status else "Access Denied")


In [16]:
from IPython.display import display

for token, phrase in waffle_tokens.items():
    tokenized = tokenizer(phrase)["input_ids"]
    display(f"{token}: {tokenized} -> {tokenizer.decode(tokenized)}")



'jelly_bottom: [101, 13988, 168, 6471, 102] -> [CLS] scrambled _ eggs [SEP]'

'hashbrowns_top: [101, 1144, 1324, 12725, 6540, 1116, 168, 2700, 102] -> [CLS] hashbrowns _ selected [SEP]'

'tomato_top: [101, 26422, 168, 2700, 102] -> [CLS] tomato _ selected [SEP]'

'napkin_top: [101, 184, 2980, 3263, 1348, 168, 2700, 102] -> [CLS] oatmeal _ selected [SEP]'

# Waffle House Formal Authentication

In [36]:
# Define Waffle House markers and expected results
waffle_tokens = {
    "jelly_bottom": "scrambled_eggs",
    "hashbrowns_top": "hashbrowns_selected",
    "tomato_top": "tomato_selected",
    "napkin_top": "oatmeal_selected"
}



# Tokenize each marker phrase
tokenized_markers = {k: tokenizer(v)["input_ids"] for k, v in waffle_tokens.items()}
pprint(tokenized_markers)


from IPython.display import display
display(tokenized_markers)


{'jelly_bottom': [101, 13988, 168, 6471, 102],
 'hashbrowns_top': [101, 1144, 1324, 12725, 6540, 1116, 168, 2700, 102],
 'tomato_top': [101, 26422, 168, 2700, 102],
 'napkin_top': [101, 184, 2980, 3263, 1348, 168, 2700, 102]}

In [37]:
def authenticate_user(tokens):
    """
    Authenticates the user based on the provided tokens.
    Each token is checked against the expected tokenized markers.
    """
    for token, expected_token in tokenized_markers.items():
        if tokens.get(token) != expected_token:
            return False
    return True

In [38]:
# Sample user tokens to authenticate
user_tokens = {
    "jelly_bottom": tokenizer("scrambled_eggs")["input_ids"],
    "hashbrowns_top": tokenizer("hashbrowns_selected")["input_ids"]
}

auth_status = authenticate_user(user_tokens)
print("Access Granted" if auth_status else "Access Denied")

user_tokens


{'jelly_bottom': [101, 13988, 168, 6471, 102],
 'hashbrowns_top': [101, 1144, 1324, 12725, 6540, 1116, 168, 2700, 102]}

### Verification Functions Based on SMV Rules

In [39]:
def check_scrambled_eggs_rule(tokens):
    """If jelly packet is at the bottom, scrambled eggs must be true"""
    return tokens.get("jelly_bottom") == tokenized_markers["jelly_bottom"]

def check_grits_rule(tokens):
    """If hashbrowns are chosen, scrambled eggs must be true, and grits must be false"""
    return (tokens.get("hashbrowns_top") == tokenized_markers["hashbrowns_top"] and
            "grits" not in tokens)

def check_oatmeal_rule(tokens):
    """If oatmeal is ordered, napkin and brown sugar must be on top"""
    return tokens.get("napkin_top") == tokenized_markers["napkin_top"]

def check_conflict_rule(tokens):
    """Prevent both hashbrowns and tomatoes from being on top at the same time"""
    return not (tokens.get("hashbrowns_top") and tokens.get("tomato_top"))

### Order Validation Function

In [40]:
def validate_order(tokens):
    """
    Validates the order by checking compliance with all defined rules.
    """
    rules = [
        check_scrambled_eggs_rule(tokens),
        check_grits_rule(tokens),
        check_oatmeal_rule(tokens),
        check_conflict_rule(tokens)
    ]
    return all(rules)

In [41]:
# Validate sample order
if validate_order(user_tokens):
    print("Order Valid - Configuration matches SMV rules")
else:
    print("Order Invalid - Conflict or Rule Violation Detected")

### Export to SMV Format for nuXmv Verification

In [42]:
def export_to_smv(tokens, filename="generated_wafflehouse.smv"):
    """
    Exports tokenized configurations to an SMV file format for verification.
    """
    with open(filename, 'w') as smv_file:
        # Write the module and variable definitions
        smv_file.write("MODULE main\nVAR\n")

        # Define all tokens and any dependent states (e.g., scrambled_eggs, grits) as boolean variables
        all_vars = list(waffle_tokens.keys()) + ["scrambled_eggs", "grits", "oatmeal"]
        for var in all_vars:
            smv_file.write(f"    {var} : boolean;\n")

        # Write INIT section: initialize all variables as false
        smv_file.write("\nINIT\n    ")
        init_conditions = " & ".join([f"!{var}" for var in all_vars])
        smv_file.write(init_conditions + ";\n\n")

        # Write TRANS section based on validation rules
        smv_file.write("TRANS\n")

        # Include TRANS conditions based on which tokens are validated
        if "jelly_bottom" in tokens:
            smv_file.write("    (jelly_bottom -> scrambled_eggs) &\n")
        if "hashbrowns_top" in tokens:
            smv_file.write("    (hashbrowns_top -> (scrambled_eggs & !grits)) &\n")
        if "napkin_top" in tokens:
            smv_file.write("    (oatmeal -> (napkin_top & brown_sugar_top));\n")

        # Write LTL specifications for formal verification
        smv_file.write("\n-- Formal Specifications\n")
        smv_file.write("LTLSPEC G (jelly_bottom -> scrambled_eggs);\n")
        smv_file.write("LTLSPEC G (oatmeal -> F(napkin_top & brown_sugar_top));\n")
        smv_file.write("LTLSPEC !(hashbrowns_top & tomato_top);\n")

    print(f"SMV model exported to {filename}")


In [43]:
# Export the tokens for formal verification
export_to_smv(user_tokens)